# Hands-on

## Setup Envs

### Config DB Engine and Pandas

In [13]:
import mysql.connector
from mysql.connector import Error
g_dbconfig = {
  'user': 'root',
  'password': 'root1MYSQL',
  'host': '127.0.0.1',
  'database': 'pymysql',
  'raise_on_warnings': True
}

myengine = mysql.connector.connect(**g_dbconfig)

In [14]:
from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine

In [15]:
alchemyconfig = {
    'user': 'root',
    'passw': 'root1MYSQL',
    'host': '127.0.0.1',
    'port': 3306,
    'database': 'pandasdb'
}

alchemyURL= 'mysql+mysqlconnector://{user}:{passw}@{host}/{database}'.format(**alchemyconfig)

mydb = create_engine(alchemyURL, echo=False)
readengine = create_engine(alchemyURL, echo=False)

In [16]:
pd.read_sql_query("show tables", mydb)

,Tables_in_pandasdb
0,dealer
1,goods
2,goods_customer
3,goods_promotion
4,goods_sell
5,past_due
6,retail_cus_ids
7,retail_transactions
8,retailcusids
9,retailtransactions


### Config Path and Envs

In [17]:
from pathlib import Path

In [18]:
data_dir = r'C:\Codes\DataScience\Data\mysql\data_capstone'
p = Path(data_dir)
csvfiles = [file for file in filter(lambda f: f.is_file() and (f.suffix == '.csv'), p.iterdir())]
dirs = [dirname for dirname in filter(lambda f: f.is_dir(), p.iterdir())]

In [61]:
stockdata_dir = r'C:\Codes\DataScience\Data\mysql\data_capstone\stk'
pstock = Path(stockdata_dir)
txtfiles = [file for file in filter(lambda f: f.is_file() and (f.suffix == '.txt'), pstock.iterdir())]

In [19]:
dirs

[WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk')]

In [62]:
txtfiles

[WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181003.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181004.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181005.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181008.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181009.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181010.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181011.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181012.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181015.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181016.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/stk/NASDAQ_20181017.txt'),
 WindowsPath('C:/Codes/DataScience/Data/mys

In [20]:
csvfiles

[WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/past_due.csv'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/session.csv'),
 WindowsPath('C:/Codes/DataScience/Data/mysql/data_capstone/TRANSACTION.csv')]

## ETL in Action

### Loading

In [44]:
# if_exists : {‘fail’, ‘replace’, ‘append’}, default ‘fail’
def load_csv(files, action='fail', block=512):
    for file in files:
        tmp_df = pd.read_csv(str(file))
        try:
            tmp_df.to_sql(name=file.stem, con=mydb, index=False, if_exists=action, chunksize=block)
            print(f'write {file.stem} to mysql!')
        except:
            df = pd.read_sql_query(f"show tables like '{file.stem}'", readengine)
            if df.empty:
                print(f'failed to write "{file.stem}" to mysql!')
            else:
                print(f'{file.stem} exists, pass loading.')
            continue

In [45]:
load_csv(csvfiles)

write past_due to mysql!
session exists, pass loading.
write TRANSACTION to mysql!


C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'TRANSACTION' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [49]:
pd.read_sql_query("select count(*) as total_row from past_due;", readengine)

,total_row
0,143712


In [ ]:
for file in txtfiles:
    tmp_df = pd.read_csv(str(file))
    try:
        tmp_df.to_sql(name=file.stem, con=mydb, index=False, if_exists=action, chunksize=block)
        print(f'write {file.stem} to mysql!')
    except:
        df = pd.read_sql_query(f"show tables like '{file.stem}'", readengine)
        if df.empty:
            print(f'failed to write "{file.stem}" to mysql!')
        else:
            print(f'{file.stem} exists, pass loading.')

## Analysis

In [58]:
143712*0.3

43113.6

In [56]:
df = pd.read_sql_query("select * from past_due where rand()<0.3;", readengine)

In [57]:
df

,id,report_date,past_due
0,Acc10,1-Feb-03,No
1,Acc123,1-Feb-03,No
2,Acc14,1-Feb-03,No
3,Acc140,1-Feb-03,No
4,Acc153,1-Feb-03,No
...,...,...,...
43090,Acc5953,26-Dec-05,No
43091,Acc5627,27-Dec-05,No
43092,Acc5796,27-Dec-05,No
43093,Acc5891,27-Dec-05,No


# Fakename ETL Examplem

In [86]:
import numpy as np
import pandas as pd

In [87]:
col_names = ['Number', 'Gender', 'Title', 'FirstName', 'LastName', 'Address', 'ZipCode',
       'EmailAddress', 'Username', 'Password', 'Birthday', 'CCType',
       'CCNumber', 'CVV2', 'CCExpires', 'BloodType', 'Kilograms',
       'Centimeters']
# col_types = {
#     'Number': np.int,
#     'Kilograms': np.float32,
#     'Centimeters': np.int

# }

In [88]:
file = 'FakeNamesUK.csv'
df = pd.read_csv(file, 
                 names=col_names,
                 error_bad_lines=False,
                 skiprows=[0],
                 encoding='latin-1')

C:\Users\hpzju\Anaconda3\envs\ds\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12,13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
file

'FakeNamesUK.csv'

In [126]:
ddf = df[  ~df['CCNumber'].isna() ]

In [135]:
dddf.to_csv('201909121139_'+file, index=False)

In [131]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99879 entries, 0 to 99995
Data columns (total 18 columns):
Number          99879 non-null float64
Gender          98732 non-null object
Title           99879 non-null object
FirstName       99879 non-null object
LastName        99879 non-null object
Address         99879 non-null object
ZipCode         99879 non-null object
EmailAddress    99879 non-null object
Username        99879 non-null object
Password        99879 non-null object
Birthday        99879 non-null object
CCType          99879 non-null object
CCNumber        99879 non-null object
CVV2            99879 non-null object
CCExpires       99879 non-null object
BloodType       99861 non-null object
Kilograms       99803 non-null object
Centimeters     99690 non-null object
dtypes: float64(1), object(17)
memory usage: 14.5+ MB


<class 'pandas.core.frame.DataFrame'>
Int64Index: 99879 entries, 0 to 99995
Data columns (total 18 columns):
Number          99879 non-null float64
Gender          98732 non-null object
Title           99879 non-null object
FirstName       99879 non-null object
LastName        99879 non-null object
Address         99879 non-null object
ZipCode         99879 non-null object
EmailAddress    99879 non-null object
Username        99879 non-null object
Password        99879 non-null object
Birthday        99879 non-null object
CCType          99879 non-null object
CCNumber        99879 non-null object
CVV2            99879 non-null object
CCExpires       99879 non-null object
BloodType       99861 non-null object
Kilograms       99803 non-null object
Centimeters     99690 non-null object
dtypes: float64(1), object(17)
memory usage: 14.5+ MB


In [104]:
df.iloc[2630:2670]

,Number,Gender,Title,FirstName,LastName,Address,ZipCode,EmailAddress,Username,Password,Birthday,CCType,CCNumber,CVV2,CCExpires,BloodType,Kilograms,Centimeters
2630,2632.0,female,Mrs.,Erin,Weston,"46 Friar Street, CLAVERTON, UK",BA2 6XF,ErinWeston@dayrep.com,Scalwat1967,Ubicofoo0,7/10/1967,MasterCard,5316316751514173,748,11/2019,A+,99.8,172
2631,2633.0,male,Mr.,Irakli,Yudin,"26 Scotswood Road, HOO, UK",IP13 3RZ,IrakliYudin@einrot.com,Sciet1973,eem8lai1C,4/4/1973,Visa,4539571363734726,571,12/2020,A+,73.9,183
2632,2634.0,female,Ms.,Jia Li,Ch'iu,"61 Layburn Court, TOTEGAN, UK",KW14 7BR,JiaLiChiu@gustr.com,Houre1986,Itheef9quee!,7/29/1986,Visa,4916071463717872,392,2/2018,O-,69.7,169
2633,2635.0,female,Ms.,Wei,Wang,"84 Ramsgate Rd, WIGGLESWORTH, UK",BD23 2DY,WeiWang@cuvox.de,Conven41,Ogh6thie,4/11/1941,Visa,4929808507312789,524,9/2020,B+,73.6,163
2634,2636.0,female,Ms.,Zhu,T'ang,"30 Marlborough Crescent, SOUTHBURN, UK",YO25 6JU,ZhuTang@superrito.com,Twomithe69,"xai3Ahnee,",5/19/1969,MasterCard,5483620258701184,197,1/2016,A+,57.0,158
2635,2637.0,male,Mr.,Yaropolk,Alexandrov,"15 Trinity Crescent, WHITBOURNE, UK",WR6 6SD,YaropolkAlexandrov@superrito.com,Capost,EiYie7oy3h,8/23/1948,Visa,4532729376496611,083,5/2018,B+,72.6,179
2636,2638.0,female,Mrs.,Anastasia,Yevseyeva,"77 Holburn Lane, HELLIFIELD, UK",BD23 8ZU,AnastasiaYevseyeva@dayrep.com,Aftyrand,Jait8Fuv4a,12/7/1936,MasterCard,5129914768426998,099,11/2016,O+,66.3,165
2637,2639.0,female,Ms.,Yun,2640,female,Ms.,Nick,2641,female,Mrs.,Sophie,2642,female,Mrs.,Greta,2643,female
2638,NaN,Ms.,Isabella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2639,2644.0,male,Mr.,Armen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
len(index)

99996

TypeError: 'bool' object is not callable

# END

In [138]:
pd.to_datetime('09/1990')

Timestamp('1990-09-01 00:00:00')